In [1]:

import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt



In [2]:

with open("../datasets/rotten_tomatoes_movies_data_with_score_panels.json", "r") as fp:
    data = json.load(fp)
    
print(len(data))


4486


In [3]:
def append_row(df, row):
    return pd.concat([
        df, 
        pd.DataFrame([row], columns=row.index)]
    ).reset_index(drop=True)



## Movie_Data_2k:

- This is smallest, yet most comprehensive version of the collected information w.r.t. to the comprehensiveness of columns related to features and network data

In [ ]:


issues = []
errors = []


movies_data = pd.DataFrame(
    columns=[
        'Title', 'Synopsis', 'Original Language', 'Runtime', 
        'Director', 'Producer', 'Writer', 'Top Cast',
        'Distributor', 'Production Co', 
        'Box Office (Gross USA)', 
        'Tomato Meter', 'Audience Score', 'No. Reviews', 'Genre', 
        'Release Date (Theaters)', 'Release Date (Streaming)', 'Link'

    ]
)



for k, v in data.items():
    try:
        a_row = pd.Series({
            'Title': v['Title'].strip(),
            'Synopsis': v['Synopsis'], 
            'Original Language': v['Info']['Original Language'], 
            'Runtime': v['Info']['Runtime'], 
            'Director': v['Info']['Director'], 
            'Producer': v['Info']['Producer'], 
            'Writer':  v['Info']['Writer'],
            'Top Cast': v["Top Cast"], 
            'Distributor': v['Info']['Distributor'],
            'Production Co': v['Info']['Production Co'],
            'Box Office (Gross USA)': v['Info']['Box Office (Gross USA)'], 
            'Tomato Meter': v["Score Panel"][2],
            'Audience Score': v["Score Panel"][5],
            'No. Reviews': int(v["Score Panel"][4].split(" ")[0]),
            'Genre': v['Info']['Genre'], 
            'Release Date (Theaters)': v['Info']['Release Date (Theaters)'],
            'Release Date (Streaming)':  v['Info']['Release Date (Streaming)'], 
            'Link': k.strip()
        })
        movies_data = append_row(df=movies_data, row=a_row)
    except Exception as error:
        print(
            f"In {k} \n"
            f"{error} \n"
            f"occurred !"
        )
        issues.append(k)
        errors.append(error)
        
        
                

In [ ]:

movies_data.shape 



In [ ]:

errors = [str(e) for e in errors]
plt.figure(figsize=(17, 5))
plt.hist(errors)
plt.show()




In [ ]:

movies_data.head()



In [ ]:


movies_data.shape



In [ ]:


movies_data.to_csv("../datasets/movies_data_2k.csv")



## Movie_Data_4k:

- The "Box office", "Distributer" , "Production Co.", "Release Date (Theaters)" and "Release Date (Streaming)" are dropped from the set of features to increase the number of data points

- The "Producer" is also drop from the corresponding network columns to increase the number of data points.

In [ ]:




issues_4k = []
errors_4k = []


movies_data_4k = pd.DataFrame(
    columns=[
        'Title', 'Synopsis', 'Original Language', 'Runtime', 
        'Director', 
        'Writer', 
        'Top Cast',
        'Tomato Meter', 'Audience Score', 'No. Reviews', 'Genre', 
        'Link'

    ]
)



for k, v in data.items():
    try:
        a_row = pd.Series({
            'Title': v['Title'].strip(),
            'Synopsis': v['Synopsis'], 
            'Original Language': v['Info']['Original Language'], 
            'Runtime': v['Info']['Runtime'], 
            'Director': v['Info']['Director'], 
#             'Producer': v['Info']['Producer'], 
            'Writer':  v['Info']['Writer'],
            'Top Cast': v["Top Cast"], 
            'Tomato Meter': v["Score Panel"][2],
            'Audience Score': v["Score Panel"][5],
            'No. Reviews': int(v["Score Panel"][4].split(" ")[0]),
            'Genre': v['Info']['Genre'], 
            'Link': k.strip()
        })
        movies_data_4k = append_row(df=movies_data_4k, row=a_row)
    except Exception as error:
        print(
            f"In {k} \n"
            f"{error} \n"
            f"occurred !"
        )
        issues_4k.append(k)
        errors_4k.append(error)
        
        
                

In [ ]:

errors_4k = [str(e) for e in errors_4k]
plt.figure(figsize=(17, 5))
plt.hist(errors_4k)
plt.show()




In [ ]:
set(errors_4k)

In [ ]:

movies_data_4k.shape



In [ ]:



movies_data_4k.to_csv("../datasets/movies_data_4k.csv")



# EDA

In [4]:

movies_data_s = pd.read_csv("../datasets/movies_data_2k.csv", )
movies_data_m = pd.read_csv("../datasets/movies_data_4k.csv", )



In [8]:
features = [
    'Title', 'Synopsis', 'Original Language', 'Runtime', 
    'Director', 'Producer', 'Writer', 'Top Cast',
    'Distributor', 'Production Co', 
    'Box Office (Gross USA)', 
    'Tomato Meter', 'Audience Score', 'No. Reviews', 'Genre', 
    'Release Date (Theaters)', 'Release Date (Streaming)', 'Link'
]

q_features = ['Runtime', 'Box Office (Gross USA)', 'Tomato Meter', 'Audience Score', 'No. Reviews', ]

c_features = ['Original Language', 'Genre',]

network_columns = [ 'Director', 'Producer', 'Writer', 'Top Cast']



In [11]:


movies_data_s.groupby["Distributor"]




TypeError: 'method' object is not subscriptable